In [1]:
import ee
import geemap
import json
import os
#from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import segment_testing_my_gee_functions as mgf

In [2]:
ee.Authenticate()
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [3]:
shp_folder_path = 'E:/!!Research/!!!Data/example_data/example_data_py/example_data_shp/'
shp_file = 'example_data.shp'
example_data = geemap.shp_to_ee(f'{shp_folder_path}{shp_file}')

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# checking that the imported layer loadedn
Map.addLayer(example_data)

In [6]:
naip = mgf.naip_func(example_data)

vis_param_naip = {'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
Map.addLayer(naip, vis_param_naip, f'naip')

In [8]:
savi = mgf.naip_savi_endvi(naip)
vis_param_savi = {'bands' : ['savi'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}

#Map.addLayer(savi, vis_param_savi, f'savi')

In [9]:
#from my_gee_functions_c import stdrd_ref_func
# Uncomment the below lines if using an item list
#poly_num = 14
#standardized = stdrd_ref_func(ee_items[poly_num])

standardized = mgf.stdrd_func(savi, example_data)
vis_param_std = {'bands' : ['N', 'R', 'G'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}

#Map.addLayer(standardized, vis_param_std, f'dunbar_park_bounds')

Testing the gaussian filter

In [12]:
#dog = mgf.dog_sharp(naip)

def gauss_func(given_image):
    ### gaussian smoothing
    gaussianKernel = ee.Kernel.gaussian(
      radius = 10, # was originally 3
      units = 'pixels'
    )
    
    gaussian_smooth = given_image.convolve(gaussianKernel)
    return gaussian_smooth

gauss = gauss_func(standardized)
vis_param_smooth = {'bands' : ['R', 'G', 'B'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
Map.addLayer(gauss, vis_param_smooth, f'smoothed')

Testing the median filter

In [21]:
uniformKernel = ee.Kernel.square(
   radius = 10,
   units = 'pixels',
)

median = standardized.reduceNeighborhood(
   reducer = ee.Reducer.median(),
   kernel = uniformKernel
)

median = median.rename(['R', 'G', 'B', 'N', 'savi', 'endvi'])
vis_param_median = {'bands' : ['R', 'G', 'B'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
Map.addLayer(median, vis_param_median, f'median')

In [14]:
edges = gauss.convolve(ee.Kernel.laplacian8())

vis_param_edge = {'bands' : ['savi'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
Map.addLayer(edges, vis_param_edge, f'edge')

conversion of color NAIP to radiance grayscale image

In [15]:
grayscale = naip.expression(
      '(0.3 * R) + (0.59 * G) + (0.11 * B)', {
      'R': naip.select(['R']),
      'G': naip.select(['G']),
      'B': naip.select(['B'])
})

vis_param_gray = {#'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
Map.addLayer(grayscale, vis_param_gray, f'gray')

In [16]:
# if wanting to use a grayscale image
int_gray = grayscale.int()
glcmTexture = int_gray.glcmTexture(5)

# if not wanting to use a grayscaled image
#int_naip = naip.int()
#glcmTexture = int_naip.glcmTexture(10)

band names for gray : \
constant_asm, constant_contrast, constant_corr, constant_var, constant_idm, constant_savg, constant_svar, constant_sent, constant_ent, constant_dvar, constant_dent, constant_imcorr1, constant_imcorr2, constant_maxcorr, constant_diss, constant_inertia, constant_shade, constant_prom

band names for rgb : \
R_asm, R_contrast, R_corr, R_var, R_idm, R_savg, R_svar, R_sent, R_ent, R_dvar, R_dent, R_imcorr1, R_imcorr2, R_maxcorr, R_diss,
R_inertia, R_shade, R_prom, \
G_asm, G_contrast, G_corr, G_var, G_idm, G_savg, G_svar, G_sent, G_ent, G_dvar, G_dent, G_imcorr1, G_imcorr2, G_maxcorr, G_diss, G_inertia, G_shade, G_prom, \
B_asm, B_contrast, B_corr, B_var, B_idm, B_savg, B_svar, B_sent, B_ent, B_dvar, B_dent, B_imcorr1, B_imcorr2, B_maxcorr, B_diss, B_inertia, B_shade, B_prom, \
N_asm, N_contrast, N_corr, N_var, N_idm, N_savg, N_svar, N_sent, N_ent, N_dvar, N_dent, N_imcorr1, N_imcorr2, N_maxcorr, N_diss, N_inertia, N_shade, N_prom

In [ ]:
contrastVis = {
   'bands' : ['r']
}

Map.addLayer(glcmTexture, contrastVis, 'contrast')

For PCA tests

In [17]:
region = example_data
image = glcmTexture

# Set some information about the input to be used later.
scale = 1
bandNames = image.bandNames()

# Mean center the data to enable a faster covariance reducer
# and an SD stretch of the principal components.
meanDict = image.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e9
})
means = ee.Image.constant(meanDict.values(bandNames))
centered = image.subtract(means)

# This helper function returns a list of new band names.
def getNewBandNames(prefix):
  seq = ee.List.sequence(1, bandNames.length())
  return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))


# This function accepts mean centered imagery, a scale and
# a region in which to perform the analysis.  It returns the
# Principal Components (PC) in the region as a new image.
def getPrincipalComponents(centered, scale, region):
  # Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar= arrays.reduceRegion(**{
    'reducer': ee.Reducer.centeredCovariance(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e9
  })

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()

  # This is a P-length vector of Eigenvalues.
  eigenValues = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigenVectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt()) \
    .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

  # Turn the PCs into a P-band image, normalized by SD.
  return principalComponents \
    .arrayProject([0]) \
    .arrayFlatten([getNewBandNames('pc')]) \
    .divide(sdImage) \


# Get the PCs at the specified scale and in the specified region
pcImage = getPrincipalComponents(centered, scale, region)



In [ ]:
pcVis = {
   'bands' : ['r']
}
Map.addLayer(pcImage, pcVis, 'pca')

In [17]:
segmented_image = mgf.segmentation_func(median, seed_spacing = 64, snic_compactness = 0, 
                                                     snic_connectivity = 4, seed_grid_shape = 'square')
Map.addLayer(segmented_image.randomVisualizer(), {}, 'clusters')

In [19]:
max_clusters = 10
obj_classed = mgf.obj_class(example_data, segmented_image, 
                                         train_scale = 1, train_pixels = 10000, #default was 10000
                                         max_clusters = max_clusters, tilescale = 5, segmented_as_bands = False, 
                                         band_list = None)
obj_palette = ['#000000','#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']
obj_vis = {'min' : 0, 'max' : 12, 'palette' : obj_palette}
Map.addLayer(obj_classed, obj_vis, f'object based classification with poly_1 and {max_clusters} clusters')

In [14]:
shp = example_data
shp_geom = shp.geometry()
task_obj = ee.batch.Export.image.toDrive(
    image = pcImage,
    description = f'pca_image',
    folder = '!imagery',
    region = shp_geom,
    scale = 1,
    #crs = 'EPSG:6350',
    maxPixels = 130000000
)
task_obj.start()